In [1]:
library(tidyverse)

── Attaching packages ───────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ──────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
data <- read_csv("Tallo.csv")

Rows: 498838 Columns: 13
── Column specification ──────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (7): tree_id, division, family, genus, species, height_outlier, crown_ra...
dbl (6): latitude, longitude, stem_diameter_cm, height_m, crown_radius_m, re...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
filtered_species <- data %>% filter(species %in% c("Fagus sylvatica", "Quercus ilex", "Pinus pinaster"))

In [4]:
count(filtered_species, species)

species,n
<chr>,<int>
Fagus sylvatica,9470
Pinus pinaster,11766
Quercus ilex,22760


In [11]:
sample(c("train","test"),size = nrow(filtered_species),prob=c(.8,.2))

ERROR: Error in sample.int(length(x), size, replace, prob): cannot take a sample larger than the population when 'replace = FALSE'


In [33]:
set.seed(42)
train_test <- filtered_species %>%
    mutate(set=sample(c("train","test"),size = n(),prob=c(.8,.2),replace=T))
train_test %>%
    count(set, species)

set,species,n
<chr>,<chr>,<int>
test,Fagus sylvatica,1961
test,Pinus pinaster,2349
test,Quercus ilex,4587
train,Fagus sylvatica,7509
train,Pinus pinaster,9417
train,Quercus ilex,18173


In [35]:
train_test %>%
    filter(set=="train") %>%
    select(species, latitude, longitude, stem_diameter_cm, height_m, crown_radius_m) %>%
    write_tsv("train.tsv")

In [36]:
train_test %>%
    filter(set=="test") %>%
    select(tree_id, latitude, longitude, stem_diameter_cm, height_m, crown_radius_m) %>%
    write_tsv("test.tsv")

In [37]:
train_test %>%
    filter(set=="test") %>%
    select(tree_id, species) %>%
    write_tsv("truth.tsv")

In [40]:
library(naniar)

In [45]:
train_test %>% group_by(set, species) %>% miss_var_summary %>% filter(n_miss>0)

species,set,variable,n_miss,pct_miss
<chr>,<chr>,<chr>,<int>,<dbl>
Fagus sylvatica,test,crown_radius_m,503,25.650178480
Fagus sylvatica,test,height_m,101,5.150433452
Fagus sylvatica,train,crown_radius_m,1883,25.076574777
Fagus sylvatica,train,height_m,432,5.753096284
Quercus ilex,train,crown_radius_m,1472,8.099928465
Quercus ilex,train,height_m,1,0.005502669
Quercus ilex,test,crown_radius_m,330,7.194244604
